# Visualize data on an interactive map

This notebook uses `lonboard` for interactive visualisation of data.

In [26]:
import geopandas as gpd
import lonboard
from lonboard.colormap import apply_continuous_cmap
import matplotlib as mpl
from mapclassify import classify

from sidecar import Sidecar

Define data path

In [10]:
chars_dir = "/data/uscuni-ulce/processed_data/chars/"

Define region

In [11]:
region = 69300

## Buildings
Load building data and ensure the geometries are all valid Polygons.

In [ ]:
buildings = gpd.read_parquet(f"{chars_dir}buildings/chars_{region}.parquet").to_crs(4326)

buildings.geometry = buildings.make_valid()

buildings = buildings[buildings.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [40]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(buildings)

CPU times: user 3.39 s, sys: 500 ms, total: 3.89 s
Wall time: 3.89 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [41]:
sc = Sidecar(title='buildings')

Create a Map object

In [42]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [43]:
with sc:
    display(m)

List avaialable columns

In [44]:
buildings.columns

Index(['index', 'id', 'geometry', 'sdbAre', 'sdbPer', 'sdbCoA', 'ssbCCo',
       'ssbCor', 'ssbSqu', 'ssbERI', 'ssbElo', 'ssbCCM', 'ssbCCD', 'stbOri',
       'mtbSWR', 'libNCo', 'ldbPWL', 'ltcBuA', 'mtbAli', 'mtbNDi', 'ltbIBD',
       'stbCeA', 'nID', 'stbSAl', 'nodeID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [64]:
column = 'stbCeA'

classifier = classify(buildings[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Tessellation

Load tessellation data and ensure the geometries are all valid Polygons.

In [68]:
tess = gpd.read_parquet(f"{chars_dir}tessellations/chars_{region}.parquet").to_crs(4326)

tess.geometry = tess.make_valid()

tess = tess[tess.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [69]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(tess)

CPU times: user 3.28 s, sys: 424 ms, total: 3.7 s
Wall time: 3.7 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [70]:
sc = Sidecar(title='tess')

Create a Map object

In [71]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [72]:
with sc:
    display(m)

List avaialable columns

In [73]:
tess.columns

Index(['enclosure_index', 'geometry', 'stcOri', 'sdcLAL', 'sdcAre', 'sscCCo',
       'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'stcSAl', 'nodeID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [88]:
column = 'stcSAl'

classifier = classify(tess[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Enclosures

Load data and ensure the geometries are all valid Polygons.

In [90]:
enc = gpd.read_parquet(f"{chars_dir}enclosures/chars_{region}.parquet").to_crs(4326)

enc.geometry = enc.make_valid()

enc = enc[enc.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [91]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(enc)

CPU times: user 273 ms, sys: 44.1 ms, total: 317 ms
Wall time: 316 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [92]:
sc = Sidecar(title='enclosures')

Create a Map object

In [93]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [94]:
with sc:
    display(m)

List avaialable columns

In [96]:
enc.columns

Index(['eID', 'geometry', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA',
       'ltkOri', 'ltkWNB', 'likWBB'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [104]:
column = 'likWBB'

classifier = classify(enc[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

/home/martin/mambaforge/envs/urban_taxonomy/lib/python3.12/site-packages/mapclassify/classifiers.py:1592: UserWarning: Not enough unique values in array to form 20 classes. Setting k to 7.
  self.bins = quantile(y, k=k)


## Streets

Load data and ensure the geometries are all valid Polygons.

In [105]:
streets = gpd.read_parquet(f"{chars_dir}streets/chars_{region}.parquet").to_crs(4326)

streets.geometry = streets.make_valid()

Create a lonboard layer

In [114]:
%%time
layer = lonboard.PathLayer.from_geopandas(streets, width_min_pixels=1)

CPU times: user 188 ms, sys: 47.5 ms, total: 235 ms
Wall time: 235 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [115]:
sc = Sidecar(title='streets')

Create a Map object

In [116]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [117]:
with sc:
    display(m)

List avaialable columns

In [118]:
streets.columns

Index(['osm_id', 'geometry', 'mm_len', 'cdsbool', 'node_start', 'node_end',
       'sdsLen', 'sssLin', 'ldsMSL', 'ldsRea', 'ldsAre', 'sisBpM', 'sdsSPW',
       'sdsSPO', 'sdsSWD', 'nID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [129]:
column = 'sdsSPW'

classifier = classify(streets[column], 'equalinterval', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Nodes

Load data and ensure the geometries are all valid Polygons.

In [130]:
nodes = gpd.read_parquet(f"{chars_dir}nodes/chars_{region}.parquet").to_crs(4326)

Create a lonboard layer

In [136]:
%%time
layer = lonboard.ScatterplotLayer.from_geopandas(nodes, radius_min_pixels=2)

CPU times: user 142 ms, sys: 0 ns, total: 142 ms
Wall time: 141 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [137]:
sc = Sidecar(title='nodes')

Create a Map object

In [138]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [139]:
with sc:
    display(m)

List avaialable columns

In [140]:
nodes.columns

Index(['x', 'y', 'mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'sddAre', 'midRea', 'midAre'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [156]:
column = 'midAre'

classifier = classify(nodes[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])